In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
movies = pd.read_excel('movie14.xls', skiprows=4)
movies.head()

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,카운트,Count,2021.0,한국,장편,드라마,개봉,권혁재,(주)필름케이
1,탈주,Escape,2024.0,한국,장편,액션,개봉,이종필,더램프(주)
2,플라이 미 투 더 문,Fly Me to the Moon,2024.0,"미국,영국",장편,"멜로/로맨스,코미디",개봉,그랙 버랜티,NaN
3,탈출: 프로젝트 사일런스,PROJECT SILENCE,2024.0,한국,장편,스릴러,개봉,김태곤,"블라드스튜디오,(주)씨제이이엔엠"
4,하이재킹,Hijack 1971,2024.0,한국,장편,"범죄,액션",개봉,김성한,(주)퍼펙트스톰필름


In [2]:
movies = movies.영화명.tolist()
movies[:5]

['카운트', '탈주', '플라이 미 투 더 문', '탈출: 프로젝트 사일런스', '하이재킹']

In [3]:
url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5

page = requests.get(url + '범죄도시')
html = BeautifulSoup(page.text, 'html.parser')

In [4]:
soup1 = html.select('#app div div div div') # 시놉시스

In [19]:
soup2 = html.select('#app div div div div .wiki-paragraph')

In [14]:
content = ''
aa =0 
for s in soup1:
    if s.text != '':
        if '시놉시스[편집]' in s.text:
            content = s.text
            break
lst = content.split('[편집]')
for i, v in enumerate(lst):
    if v[-4:] == '시놉시스':
        synopsys = lst[i+1]

In [15]:
synopsys

'오늘 밤, 다 쓸어버린다!2004년 서울…하얼빈에서 넘어와 단숨에 기존 조직들을 장악하고가장 강력한 세력인 춘식이파 보스 ‘황사장(조재윤 분)’까지 위협하며도시 일대의 최강자로 급부상한 신흥범죄조직의 악랄한 보스 ‘장첸(윤계상 분)’.대한민국을 뒤흔든 ‘장첸(윤계상 분)’ 일당을 잡기 위해오직 주먹 한방으로 도시의 평화를 유지해 온 괴물형사 ‘마석도(마동석 분)’와인간미 넘치는 든든한 리더 ‘전일만(최귀화 분)’ 반장이 이끄는 강력반은나쁜 놈들을 한방에 쓸어버릴 끝.짱.나.는. 작전을 세우는데…통쾌하고! 화끈하고! 살벌하게!올 추석, 나쁜 놈들 때려잡는 강력반 형사들의 ‘조폭소탕작전’이 시작된다!4. 등장인물'

In [29]:
lst = [] # 영화에 대한 정보를 담는 리스트
st = 0
for s in soup2[1:]:
    if s.text != '':
        if s.text == '장르':
            st = 1
        elif '개요' in s.text:
            st = 0
            break
        if st == 1:
            lst.append(s.text)
    if '개요' in s.text:
        break

In [30]:
lst

['장르',
 '범죄, 액션, 스릴러, 느와르, 블랙 코미디, 형사',
 '감독',
 '강윤성',
 '각본',
 '기획',
 '마동석',
 '제작',
 '마동석, 김홍백, 장원석',
 '각색',
 '이석근',
 '출연',
 '마동석, 윤계상, 조재윤, 최귀화 外',
 '촬영',
 '주성림',
 '편집',
 '김선민',
 '음악',
 '모그',
 '미술',
 '김성규',
 '소품',
 '진훈',
 '세트',
 '김덕두',
 '의상',
 '김경미',
 '분장',
 '남지수',
 '무술',
 '허명행, 전재형',
 '조명',
 '최종하',
 '동시녹음',
 '전영기',
 '특수효과',
 '윤대원',
 '사운드',
 '공태원',
 '촬영 기간',
 '2017년 2월 27일 ~ 2017년 7월 18일',
 '제작사',
 '(주)홍필름 | 비에이엔터테인먼트',
 '배급사',
 ' 메가박스㈜플러스엠 | ㈜키위미디어그룹',
 '개봉일',
 ' 2017년 10월 3일',
 ' [ 펼치기 · 접기 ]  2017년 10월 6일  2017년 11월 2일 2017년 12월 11일 (국제 영화제 및 마카오 시상식) 2017년 12월 15일 2018년 1월 4일 2018년 2월 1일 2018년 4월 12일 2018년 4월 28일 2018년 9월 8일 (한씨네 한국영화제) 2018년 10월 2일 ( CPH:PIX 영화제) 2018년 10월 3일 (어버더 숀스 영화제) 2020년 2월 20일',
 '화면비',
 '2.35:1',
 '상영 타입',
 '2D',
 '상영 시간',
 '121분 (2시간 1분)',
 '제작비',
 '70억 원',
 '월드 박스오피스',
 '$52,097,753 (최종)',
 '대한민국 총 관객 수',
 '6,880,546명 (최종)',
 '스트리밍',
 'Disney+ ▶TVING ▶▶',
 '상영 등급',
 '  청소년 관람불가',
 ' ']

# 기존 작업

In [ ]:
pd_movies = []
for movie in movies:
    try:
        url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5

        page = requests.get(url + movie)
        html = BeautifulSoup(page.text, 'html.parser')
        # html.select('#app > div > div.MMT57cZa.iHNYoQlC > div > div')
        soup = html.select('#app div div div div .wiki-paragraph') # .wiki-paragraph

        time.sleep(1) # 1초동안 정지
        
        lst = [] # 영화에 대한 정보를 담는 리스트
        st = 0
        for s in soup:
            if s.text != '':
                if s.text == '장르':
                    st = 1
                elif '개요' in s.text:
                    st = 0
                    break
                if st == 1:
                    lst.append(s.text)
            if '개요' in s.text:
                break
        lst.insert(0, '영화명')
        lst.insert(1, movie)
        df = pd.DataFrame(lst)
        df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
        df.columns = ['kind', 'content']
        df.set_index(df.kind, inplace=True)
        df = df.drop('kind', axis=1)
        pd_movies.append(df)
    except Exception as e:
        print(e)

In [58]:
pd_movies[5]

,content
kind,
영화명,우마무스메 프리티 더비 새로운 시대의 문
장르,"경마, 스포츠"
감독,야마모토 켄
시나리오 디렉터,코바리 테츠야(小針哲也)
스토리 원안,콘티
야마모토 켄이시이 토시마사[1]나카야마 나오야[L],연출
야마모토 켄후치모토 슈헤이쿠로야나기 토시마사랴오 청즈나카야마 나오야[L]오카모토 마나부연출 보좌,각본
요시무라 키요코(吉村清子),원작
Cygames,제작


In [ ]:
# rmlist = ['기획'] # , '촬영', '편집', '음악', '미술', '분장','마케팅'
# for i in rmlist:
#     try:
#         lst.remove(i)
#     except:
#         pass